# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Using intermediary variables from workflow objects. In this workflow variables are directly used for the next step. 


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0007739' HD
    - 'HGNC:4851' Htt
    - 'CHEBI:18248' Iron (not working)
    - 'HGNC:18229' Rhes (RASD2)
    
Possible seed nodes:
https://monarchinitiative.org/search/Iron
* Connecting paths: query templates.

In [1]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils
import pandas as pd

In [2]:
%%time
# initiate empty curation network:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")
curation_nodes = curation_nodes.astype('object')
curation_nodes['name'] = ["NaN", "NaN"]

CPU times: user 5.18 ms, sys: 9.49 ms, total: 14.7 ms
Wall time: 228 ms


In [3]:
%%time
# transcriptomics:
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path)
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes = transcriptomics.build_nodes(rna_network)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/Structured review/HD-SR/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (3209, 6)
* These are the 

In [4]:
%%time
# regulation:
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)


The function "prepare_msigdb_data()" is running...

* Number of Transcription Factor Targets (TFT) gene sets: 615

The MSigDB raw network is saved at: /home/karolis/Structured review/HD-SR/bioknowledge-reviewer/bioknowledge_reviewer/regulation/msigdb/out/tf_genelist_entrez_msigdb.json. Other reporting files are also saved at the same directory.


Finished prepare_msigdb_data().


The function "load_tf_gene_edges()" is running...

Finished load_tf_gene_edges().


The function "get_gene_id_normalization_dictionaries()" is running...

* Querying BioThings to map gene symbols to HGNC and Entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
48 input query terms found no hit:
	['COMP1', 'TCF1P', 'CDPCR1', 'CREBP1', 'NKX62', 'HAND1E47', 'CACBINDINGPROTEIN', 'TAXCREB', 'CEBPGAM
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/Structure

In [5]:
%%time
# Monarch extraction:
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
    'REACT:R-HSA-917937' #Iron uptake pathway
] 

# get first degree neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print("Nodes 1st degree neighbours:")
print(len(neighboursList))

seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 1st shell nodes:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

node_list = sum([seedList, neighboursList, seed_orthophenoList, neighbours_orthophenoList], 
               [])

monarch_network = monarch.extract_edges(node_list)
print("connections after 2nd degree:")
print(len(monarch_network))
monarch.print_network(monarch_network, 'monarch_connections')


# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 4/4 [00:24<00:00,  5.44s/it]



Finished get_neighbours_list().

Nodes 1st degree neighbours:
1024

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 16/16 [00:47<00:00,  4.03s/it]



Finished get_orthopheno_list().

227

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|██████████| 7307/7307 [6:18:03<00:00,  2.44s/it]   



Finished get_orthopheno_list().

12978


NameError: name 'NeighbourList' is not defined

In [13]:
monarch_network = monarch.extract_edges(node_list)
print("connections after 2nd degree:")
print(len(monarch_network))
monarch.print_network(monarch_network, 'monarch_connections')


# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())


The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


100%|██████████| 13984/13984 [16:34:24<00:00,  5.16s/it]   



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

connections after 2nd degree:
233148

Saving Monarch edges at: '/home/karolis/Structured review/HD-SR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_v2020-12-02.csv'...


The function "build_edges()" is running...
df (233148, 9)

* This is the size of the edges file data structure: (233148, 9)
* These are the edges attributes: Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
* This is the first record:
                    object_id property_description    property_id  \
0  ENSEMBL:ENSRNOG00000046761                   NA  RO:HOM0000017   

                   property_label  \
0  in orthology relationship with   

                                   property_uri reference_date  \
0  http://purl.obolibrary.org/

In [14]:
# construct df's
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)


The function "build_edges()" is running...

Preparing networks...
Curated:
(2, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(233148, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Transcriptomics:
(3209, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['object_id', 'property_description', 'property_id', 'property_label',
       'property_uri', 'reference_date', 'reference_supporting_text',
       'reference_uri', 'subject_id'],
      dtype='object')

Concatenatin

In [15]:
# print type of objects
print('type edges:', type(edges))
print('type nodes:', type(nodes))
print()

# print objects sizes
print('len edges:', len(edges))
print('len nodes:', len(nodes))
print()

# print object attribute
print('attribute edges:', edges.columns)
print('attribute nodes:', nodes.columns)

type edges: <class 'pandas.core.frame.DataFrame'>
type nodes: <class 'pandas.core.frame.DataFrame'>

len edges: 433601
len nodes: 33817

attribute edges: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
attribute nodes: Index(['id', 'semantic_groups', 'preflabel', 'synonyms', 'name',
       'description'],
      dtype='object')


In [16]:
%%time
# create a Neo4j server instance
neo4j_dir = neo4jlib.create_neo4j_instance('3.5.6')
print('The name of the neo4j directory is {}'.format(neo4j_dir))

# import to graph database
## prepare the graph to neo4j format
edges_df = utils.get_dataframe(edges)
nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = './{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

## import graph into neo4j database
neo4jlib.do_import(neo4j_path)

Creating a Neo4j community v3.5.6 server instance...
Configuration adjusted!
Neo4j v3.5.6 is running.
The name of the neo4j directory is neo4j-community-3.5.6
statements:  433601
concepts:  33817

File './neo4j-community-3.5.6/import/ngly1/ngly1_statements.csv' saved.

File './neo4j-community-3.5.6/import/ngly1/ngly1_concepts.csv' saved.

The function "do_import()" is running...

The graph is imported into the server. Neo4j is running.You can start exploring and querying for hypothesis. If you change ports or authentication in the Neo4j configuration file, the hypothesis-generation modules performance, hypothesis.py and summary.py, will be affected.

CPU times: user 8.77 s, sys: 494 ms, total: 9.27 s
Wall time: 1min 3s
